# Models for Regression

You must

0. Perform an exploratory analysis of the data and pre-process the predictors to remove potential skewness in their distribution. Based on the transformed predictors, comment on the presence of relationships between pairs of predictors (estimate the correlation matrix). Also, are the relationships between the predictors and outcome individually linear (estimate all predictor-outcome correlations)?

Then, you must

1. Use the transformed predictors in the training set to learn an ordinary linear square (OLS) regression model and test the model using the test set (remember to apply the same pre-processing you used on the training set). Code/implement the OLS model from scratch and compare the results obtained with the build-in functions available in the programming language of your choice. Compare the model performance obtained on the test set with the estimates you would obtain using a resampling scheme as 5- or 10-fold cross validation: use both the RMSE and $R^2$ (again, code/implement the cross-validation from scratch and compare the results with those obtained with the build-in function).

2. Use the transformed predictors in the training set to learn a L2 (ou L1)-penalised linear regression model and test the model using the test set (remember to apply the same pre-processing you used on the training set). Code/implement the penalized model from scratch and compare the results obtained with the build-in functions available in the programming language of your choice. Determine the optimal value of $\lambda$ using a 5- or 10-fold cross-validation based on the RMSE (you can only use the training set in this phase, and your search space $\lambda$ should consist of at least 10 values). Report on process (show the cross-validation profile, both on terms of the RMSE and $R^2$). Report the accuracy (RMSE and $R^2$) obtained on the test set (again, code/implement the cross-validation from scratch and compare the results with those obtained with the build-in function).

3. Use the transformed predictors in the training set to learn either a PLS or a PCR regression model and test the model using the test set (remember to apply the same pre-processing you used on the training set). Determine the optimal number of components using a 5- or 10-fold cross-validation based on the RMSE (you can only use the training set in this phase). Report on process (show the cross-validation profile, both in terms of the RMSE and $R_2$). Report the accuracy (RMSE and $R_2$) obtained on the test set.

4. Use the transformed predictors in the training set to build a neural network model for regression and test the model using the test set (remember to apply the same pre-processing you used on the training set). Report the accuracy (RMSE and $R_2$) obtained on the test set. Do the nonlinear model outperform the linear model you previously developed? If so, what might this tell you about the underlying relationship between the predictors and the response?

## Importing packages

In [1]:
import sys

sys.path.append("../../")

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from os.path import join
import numpy as np

In [3]:
from ica.dataset import preprocess_regression_dataset

2024-12-30 07:52:09.717 | INFO     | ica.config:<module>:11 - PROJ_ROOT path is: /mnt/c/Users/franc/Code/UFC/MSc/S01/ica


In [4]:
pd.options.display.max_columns = None

sns.set_theme(style='darkgrid')
sns.set_palette("twilight_shifted")

## Reading Database

In [5]:
data = pd.read_csv("../../data/external/Agregado-20241114133303.csv", 
                   encoding="latin-1",
                   sep=";")

print(data.shape)
data.head()

(659, 229)


,Código do Município,Município,Estado,Ano de Referência,Código do Prestador,Prestador,Sigla do Prestador,Abrangência,Tipo de Serviço,Natureza Jurídica,G05A - Quantidade total de municípios atendidos com abastecimento de água,G05B - Quantidade total de municípios atendidos com esgotamento sanitário,G06A - População urbana residente do(s) município(s) com abastecimento de água,G06B - População urbana residente do(s) município(s) com esgotamento sanitário,"G12A - População total residente do(s) município(s) com abastecimento de água, segundo o IBGE","G12B - População total residente do(s) município(s) com esgotamento sanitário, segundo o IBGE",GE001 - Quantidade de municípios atendidos com abastecimento de água com delegação em vigor,GE002 - Quantidade de municípios atendidos com abastecimento de água com delegação vencida,GE003 - Quantidade de municípios atendidos com abastecimento de água sem delegação,GE005 - Quantidade total de municípios atendidos,GE008 - Quantidade de Sedes municipais atendidas com abastecimento de água,GE009 - Quantidade de Sedes municipais atendidas com esgotamento sanitário,GE010 - Quantidade de Localidades (excluídas as sedes) atendidas com abastecimento de água,GE011 - Quantidade de Localidades (excluídas as sedes) atendidas com esgotamento sanitário,GE014 - Quantidade de municípios atendidos com esgotamento sanitário com delegação em vigor,GE015 - Quantidade de municípios atendidos com esgotamento sanitário com delegação vencida,GE016 - Quantidade de municípios atendidos com esgotamento sanitário sem delegação,GE017 - Ano de vencimento da delegação de abastecimento de água,GE018 - Ano de vencimento da delegação de esgotamento sanitário,"GE025 - Quantidade de municípios não atendidos com abastecimento de água, mas com delegação em vigor","GE026 - Quantidade de municípios não atendidos com abastecimento de água, mas com delegação vencida para prestar esses serviços","GE028 - Quantidade de municípios não atendidos com esgotamento sanitário, mas com delegação em vigor para prestar esses serviços","GE029 - Quantidade de municípios não atendidos com esgotamento sanitário, mas com delegação vencida para prestar esses serviços",AG001 - População total atendida com abastecimento de água,AG001A - População total atendida com abastecimento de água no ano anterior ao de referência.,AG002 - Quantidade de ligações ativas de água,AG002A - Quantidade de ligações ativas de água no ano anterior ao de referência.,AG003 - Quantidade de economias ativas de água,AG003A - Quantidade de economias ativas de água no ano anterior ao de referência.,AG004 - Quantidade de ligações ativas de água micromedidas,AG004A - Quantidade de ligações ativas de água micromedidas no ano anterior ao de referência.,AG005 - Extensão da rede de água,AG005A - Extensão da rede de água no ano anterior ao de referência.,AG006 - Volume de água produzido,AG007 - Volume de água tratada em ETAs,AG008 - Volume de água micromedido,AG010 - Volume de água consumido,AG011 - Volume de água faturado,AG012 - Volume de água macromedido,AG013 - Quantidade de economias residenciais ativas de água,AG013A - Quantidade de economias residenciais ativas de água no ano anterior ao de referência.,AG014 - Quantidade de economias ativas de água micromedidas,AG014A - Quantidade de economias ativas de água micromedidas no ano anterior ao de referência.,AG015 - Volume de água tratada por simples desinfecção,AG016 - Volume de água bruta importado,AG017 - Volume de água bruta exportado,AG018 - Volume de água tratada importado,AG019 - Volume de água tratada exportado,AG020 - Volume micromedido nas economias residenciais ativas de água,AG021 - Quantidade de ligações totais de água,AG021A - Quantidade de ligações totais de água no ano anterior ao de referência.,AG022 - Quantidade de economias residenciais ativas de água micromedidas,AG022A - Quantidade de economias residenciais ativas de água micromedidas no ano anterior ao de referência.,AG024 - Volume de serviço,AG025A - População

In [6]:
df = preprocess_regression_dataset(data)
df.head()

,nm_municipio,ano,abrangencia,tipo_servico,nat_juridica,IN001,IN009,IN020,IN022,IN023,IN037,IN038,IN055,IN057
0,Acopiara,2022,Microrregional,Água,Organização social,1.0,100.00,37.69,47.89,NaN,54.21,18.39,20.76,0.0
1,Aiuaba,2022,Local,Água,Administração pública direta,1.0,90.95,33.41,60.66,NaN,38.60,2.63,65.39,0.0
2,Aiuaba,2021,Local,Água,Administração pública direta,1.0,45.72,30.18,53.93,100.00,40.04,2.69,59.03,0.0
3,Aiuaba,2020,Local,Água,Administração pública direta,1.0,0.00,12.77,94.83,100.00,19.23,9.66,58.31,0.0
4,Aiuaba,2019,Local,Água,Administração pública direta,1.0,0.00,10.65,177.28,98.26,25.45,1.49,57.78,0.0


In [7]:
numerical_columns = ["IN001", "IN009", "IN020", "IN023", "IN037", "IN038", "IN055", "IN057"]
# categorical_columns = ["tipo_servico"]
target_col = "IN022"

## 0. Perform an exploratory analysis of the data and pre-process the predictors

Already done in HW1

### ETL of features for Regression Model

In [8]:
_df = df.dropna()
train = _df.query("ano<=2017")[numerical_columns + [target_col]]
test = _df.query("ano>2017")[numerical_columns + [target_col]]

In [9]:
X_train, y_train = train[numerical_columns], train[target_col]
X_test, y_test = test[numerical_columns], test[target_col]

In [10]:
print("The train-test ratio is: %.2f-%.2f" % (len(train)/len(_df), len(test)/len(_df)))

The train-test ratio is: 0.72-0.28


In [11]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, StandardScaler

In [12]:
ohe = make_column_transformer(
  (StandardScaler(), numerical_columns),
  # (OneHotEncoder(handle_unknown = 'ignore', sparse_output=False, dtype=float), categorical_columns),
  remainder='passthrough')

ohe.fit(X_train)
X_transformed = ohe.transform(X_train)

dummies_columns = [x.split("__")[1] for x in ohe.get_feature_names_out()]
dummies = pd.DataFrame(X_transformed, columns=dummies_columns)

X_train = dummies.astype({c: float for c in dummies_columns})

print(X_train.shape)
X_train.head()

(304, 8)


,IN001,IN009,IN020,IN023,IN037,IN038,IN055,IN057
0,-0.009544,-3.154410,0.156229,-0.083943,-1.757057,-1.260821,-2.364062,-1.067246
1,3.471937,-2.883988,0.118451,0.384630,0.428034,-0.882508,-0.297863,-1.067246
2,1.344365,-2.897836,0.052750,0.483276,0.913441,-1.072552,-0.688280,-1.067246
3,0.570703,-2.893989,0.003474,0.483276,0.691344,-1.260821,-0.963611,-1.067246
4,-0.396375,-3.154410,-0.012951,0.483276,2.915359,1.193772,-0.549886,1.222113


In [13]:
X_transformed = ohe.transform(X_test)

dummies = pd.DataFrame(X_transformed, columns=dummies_columns)

X_test = dummies.astype({c: float for c in dummies_columns})

print(X_test.shape)
X_test.head()

(118, 8)


,IN001,IN009,IN020,IN023,IN037,IN038,IN055,IN057
0,-0.589790,-1.395705,3.301670,0.483276,1.298865,-0.783045,-0.675169,-1.067246
1,-0.589790,-3.154410,0.442029,0.483276,-0.289390,0.454908,-0.710132,-1.067246
2,-0.589790,-3.154410,0.093813,0.268719,0.185331,-0.996179,-0.735868,-1.067246
3,-0.202959,-3.154410,0.129949,0.336539,-1.757057,-0.941120,-0.720329,-1.067246
4,-0.589790,0.692276,-0.950835,0.215697,0.196780,-1.124060,-1.610907,-1.067246



## 1. OLS Regression

In [14]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, root_mean_squared_error

In [15]:
type(y_train) in [pd.Series, pd.DataFrame]

True

In [16]:
##### Implementation of Ordinary Least Squares from scrach

class OLSRegressor(BaseEstimator, RegressorMixin):
    def __init__(self):
        ...

    def fit(self, X, y):
        
        if type(X) in [pd.Series, pd.DataFrame]:
            X = X.copy().values
        
        if type(y) in [pd.Series, pd.DataFrame]:
            y = y.copy().values

        # Simple fit method that calculates the mean of the target
        self.X = np.hstack((np.ones([X.shape[0],1], X.dtype), X))
        self.y = y
        self.N, self.p = X.shape
        
        self.weights = np.linalg.inv(self.X.T @ self.X) @ self.X.T @ y
        
        return self

    def predict(self, X):
        if type(X) in [pd.Series, pd.DataFrame]:
            X = X.copy().values
            
        _X = np.hstack((np.ones([X.shape[0],1], X.dtype), X))
        return _X @ self.weights
    
    def get_variance(self):
        y_hat = self.predict(self.X)
        return 1/(self.N - self.p - 1) * np.sum((y_hat - self.y)**2)
    
    def get_params_covariance(self) -> np.ndarray:
        """The variance–covariance matrix of the least squares parameter estimates

        Returns:
            np.ndarray: The variance–covariance matrix
        """
        return np.linalg.inv(self.X.T @ self.X) * self.get_variance()

### Model Implementation Comparison

In [17]:
rmse = []
r2 = []

In [18]:
sk_lm = LinearRegression()
sk_lm.fit(X_train, y_train)

sk_yhat = sk_lm.predict(X_test)
rmse.append(root_mean_squared_error(y_test, sk_yhat))
r2.append(r2_score(y_test, sk_yhat))

In [19]:
print("Coefficients: ", sk_lm.coef_)
print("Intercept: ", sk_lm.intercept_)

Coefficients:  [  7.23603742 -23.29646907   7.77149884   2.84777851  -4.39516898
  -4.52761228   4.16566111  -0.35063749]
Intercept:  128.7498355263158


In [20]:
custom_lm = OLSRegressor()
custom_lm.fit(X_train, y_train)

custom_yhat = custom_lm.predict(X_test)
rmse.append(root_mean_squared_error(y_test, custom_yhat))
r2.append(r2_score(y_test, custom_yhat))

In [21]:
print("Coefficients: ", custom_lm.weights[1:])
print("Intercept: ", custom_lm.weights[0])

Coefficients:  [  7.23603742 -23.29646907   7.77149884   2.84777851  -4.39516898
  -4.52761228   4.16566111  -0.35063749]
Intercept:  128.74983552631576


In [22]:
pd.DataFrame([rmse, r2], columns=["RMSE", "R2"], index=["Scikit-Learn", "Custom Model"])

,RMSE,R2
Scikit-Learn,75.619308,75.619308
Custom Model,-0.222027,-0.222027


### Cross-Validation Comparison

In [23]:
from sklearn.base import clone

In [24]:
k = 3

In [25]:
def split_train_test_folds(folds, i):
    return folds[:i] + folds[i+1:], folds[i]

In [26]:
def custom_cross_validate(clf, X, y, k):
    
    # Copy X into an np.array
    if type(X) in [pd.Series, pd.DataFrame]:
        X = X.copy().values
    
    # Copy y into an np.array
    if type(y) in [pd.Series, pd.DataFrame]:
        y = y.copy().values

    # Get the length of the dataset
    n = len(X)
    # Get the number of instances of the folds
    part_size = n // k

    # Splits the data in K folds and if n % k != 0, it adds the remainder in the final fold
    folds_X = [X[(i*part_size):(i*part_size+part_size)] if i < (k-1) else X[(i*part_size):] for i in range(k)]
    folds_y = [y[(i*part_size):(i*part_size+part_size)] if i < (k-1) else y[(i*part_size):] for i in range(k)]

    models = []
    rmse = []
    r2 = []
    
    for i in range(k):
        # Gets the train-validate folds
        X_train_fold, X_validation = split_train_test_folds(folds_X, i)
        y_train_fold, y_validation = split_train_test_folds(folds_y, i)

        # Concatenate the folds to get an np.array
        X_train = np.vstack(X_train_fold)
        y_train = np.hstack(y_train_fold)

        # This ensure that every time a model is trained, it's new and conntains the same parameters
        model = clone(clf)

        model.fit(X_train, y_train)
        y_hat = model.predict(X_validation)

        models.append(model)
        rmse.append(root_mean_squared_error(y_validation, y_hat))
        r2.append(r2_score(y_validation, y_hat))

    best_model_index = np.argmin(rmse)
    metrics = {"RMSE": rmse, "R2": r2}
    # Returns the best model and the metrics of each fold
    return models[best_model_index], metrics


In [27]:
lm = LinearRegression()
model, metrics = custom_cross_validate(lm, X_train, y_train, k=4)
metrics

{'RMSE': [58.95846133238152,
  92.47267660205691,
  50.3426303260102,
  58.4879286021674],
 'R2': [0.035547354671426556,
  -0.12379756140046183,
  0.036793255488130616,
  -1.342524983889244]}

In [28]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [29]:
cv = KFold(n_splits=4)
scores = cross_val_score(lm, X_train, y_train, cv=cv, scoring="neg_root_mean_squared_error")
[-score for score in scores]

[58.95846133238152, 92.47267660205691, 50.3426303260102, 58.4879286021674]